In [1]:
#Always
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import time

#Sometimes
import random
import pickle as pkl
import igraph as ig
import hypernetx as hnx
import xgi

#My codes
import Hypergraph_Models as hm
import Hypergraph_Functions as hf
import Hypergraph_Processes as hp

# Copying the diffusion function from my other notebook

In [2]:
def diffusion(w,E,epsilon,slice=1,want='numbers'):
    uniform = [sum(w.values())/len(w)]*len(w)
    distance = sp.stats.wasserstein_distance(list(w.values()),uniform)
    results = {}
    if want=='numbers':
        results[0] = distance
    else:
        results[0] = w.copy()
    round = 0
    if type(epsilon) is int:
        for i in range(epsilon-1):
            chosen_edges = random.choices(E,k=slice)
            for e in chosen_edges:
                try:
                    new_weight = sum(w[v] for v in set(e))/len(set(e))
                except:
                    print('found vertex with no weight')
                    print(e)
                    print(set(e).intersection(set(w.keys())))
                for v in set(e):
                    w[v] = new_weight
            distance = sp.stats.wasserstein_distance(list(w.values()),uniform)
            round += slice
            if want=='numbers':
                results[round] = distance
            else:
                results[round] = w.copy()
    else:
        while distance>epsilon:
            chosen_edges = random.choices(E,k=slice)
            for e in chosen_edges:
                new_weight = sum(w[v] for v in set(e))/len(set(e))
                for v in set(e):
                    w[v] = new_weight
            distance = sp.stats.wasserstein_distance(list(w.values()),uniform)
            round += slice
            if want=='numbers':
                results[round] = distance
            else:
                results[round] = w.copy()
    return results

# Getting and cleaning all of the datasets

In [3]:
filenames = [
    "contact-primary-school", 
    "contact-high-school", 
    "hospital-lyon", 
    "email-enron", 
    "email-eu", 
    "diseasome", 
    "disgenenet", 
    "ndc-substances", 
    "congress-bills", 
    "tags-ask-ubuntu",
]
datasets = {}
vertex_data = {}
edge_data = {}
for filename in filenames:
    H = xgi.load_xgi_data(filename)
    E = list(set([tuple(sorted(e)) for e in H.edges.members() if len(e)<=11 and len(e)>=2])) ## changed 11 to 5
    E = [set(e) for e in E]
    V = list(set([i for e in E for i in e]))
    
    #For ask-ubuntu, we make two smaller graphs
    #One only keeps the first 20000 edges
    #The other throws away half of the vertices
    if filename == "tags-ask-ubuntu": 
        #Edge-chopped version
        V_1 = V.copy()
        E_1 = E[:20000]
        V_1,E_1 = hf.giant_component(V_1,E_1)
        datasets["ubuntu (edge-chopped)"] = (V_1,E_1)
        vertex_data["ubuntu (edge-chopped)"] = len(V_1)
        edge_data["ubuntu (edge-chopped)"] = len(E_1)

        #Vertex-chopped version
        V_2 = []
        for v in V:
            if random.choice([1,2])==1:
                V_2.append(v)
        #We use the strict definition of induced subgraph
        E_2 = []
        for e in E:
            accept = True
            for v in e:
                if v not in V_2:
                    accept = False
                    break
            if accept:
                E_2.append(e)
        V_2,E_2 = hf.giant_component(V_2,E_2)
        datasets["ubuntu (vertex-chopped)"] = (V_2,E_2)
        vertex_data["ubuntu (vertex-chopped)"] = len(V_2)
        edge_data["ubuntu (vertex-chopped)"] = len(E_2)
    #All other datasets are built normally
    else:
        V,E = hf.giant_component(V,E)
        datasets[filename] = (V,E)
        vertex_data[filename] = len(V)
        edge_data[filename] = len(E)

df = pd.DataFrame({'n': vertex_data.copy(), 'm': edge_data.copy()})

KeyboardInterrupt: 

Verifying that all of the datasets are built correctly

In [ ]:
for name in datasets.keys():
    print(name)
    print('|V| = {0}, |E| = {1}\n'.format(len(datasets[name][0]),len(datasets[name][1])))

# Setting parameters for all experiments

In [5]:
#Whether or not we include multisets should be consistent across all experiments
multisets = False
skeleton = True

#The number of samples for each experiment will be different
#For example: adversarial growth takes forever so should only be run a few times
rolls = {
    'random growth': 100,
    'adversarial growth': 20,
    'single-source diffusion': 100,
    '10% sprinkled diffusion': 100,
}

# Getting the fitted parameters

In [6]:
d = {}
m = {}
q_list = [0, 0.5, 1]

for name in datasets.keys():
    V,E = datasets[name]

    #####
    # d #
    #####
    d[name] = hf.degrees(V,E)

    #####
    # m #
    #####
    E_sorted = hf.sort_edges(E)
    m[name] = {}
    for k in E_sorted.keys():
        m[name][k] = len(E_sorted[k])

In [7]:
t_start = time()

# Experiment 1: random growth
We start at round 0 with no edges and a max component size of 1

On round i+1, we choose a random edge (not already added) and add it to the graph, then track the max component size

We save the data as the dictionary (round -> size of giant)

In [ ]:
#This is what will get added to the dataframe
experiment = {}

#Parameters were set at the beginning
num_rolls = rolls['random growth']


limited_names = [
    "hospital-lyon", 
    "disgenenet", 
]

for name in datasets.keys():
    V,E = datasets[name]
    results = hp.giant_component_growth(V, E)
    for i in range(num_rolls - 1):
        sample = hp.giant_component_growth(V, E)
        for j in results.keys():
            results[j] += sample[j]
    for j in results.keys():
        results[j] = results[j]/num_rolls
    experiment[name] = results.copy()

    for q in q_list: 
        E = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
        results = hp.giant_component_growth(V,E)
        for i in range(num_rolls-1):
            E = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
            sample = hp.giant_component_growth(V,E)
            for j in results.keys():
                results[j] += sample[j]
        for j in results.keys():
            results[j] = results[j]/num_rolls
    
        #Saving the results
        experiment['{0} (q={1})'.format(name, q)] = results.copy()

#Saving to the dataframe
df_add = pd.DataFrame({'random growth':experiment.copy()})
df = pd.concat([df,df_add],axis=1)

Verifying that the experiment was compiled correctly

In [ ]:
df

In [ ]:
with open('fixed_experiments.pkl','wb') as file:
    pkl.dump(df, file)

# Experiment 2: adversarial growth
From the input graph, we compute the betweenness value of each edge

We start at round 0 with no edges and a max component size of 1

At round i+1, we pick the edge with smallest betweenness from the set of edges not chosen yet and add it to the graph, tracking the size of the giant

We save the data as the dictionary (round -> size of giant)

In [ ]:
##
#Francois' speed-up, (thanks for this!)
##

#Simple function to reorder edges by betweenness
#Smallest to largest
def reorder_by_betweenness_fast(edges):
    E = edges.copy()
    G = hnx.Hypergraph(E)
    LG = G.get_linegraph()
    g = ig.Graph.from_networkx(LG)
    bet_ig = g.betweenness()
    n = g.vcount()
    norm = 2/((n-1)*(n-2))
    betweenness = dict(zip(g.vs['_nx_name'],[x*norm for x in bet_ig])) 
    E_arranged = list(sorted(betweenness, key=betweenness.get))
    E = [E[i] for i in E_arranged]
    return E

In [ ]:
##
#Now the experiment
##

#This is what gets added to the dataframe
#Note that I save a copy of 'experiment' to the dataframe, so re-initializing it is fine
experiment = {}

#Parameters were set at the beginning
num_rolls = rolls['adversarial growth']

for name in limited_names:
    V,E = datasets[name]
    E = reorder_by_betweenness_fast(E)
    results = hp.giant_component_growth(V, E, shuffle_edges=False)
    experiment[name] = results.copy()

    for q in q_list: 
        E = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
        E = reorder_by_betweenness_fast(E)
        results = hp.giant_component_growth(V, E, shuffle_edges=False)
        for i in range(num_rolls-1):
            E = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
            E = reorder_by_betweenness_fast(E)
            sample = hp.giant_component_growth(V, E, shuffle_edges=False)
            for j in results.keys():
                results[j] += sample[j]
        for j in results.keys():
            results[j] = results[j]/num_rolls
    
        #Saving the results
        experiment['{0} (q={1})'.format(name, q)] = results.copy()
    
    
#Saving to the dataframe
df_add = pd.DataFrame({'adversarial growth':experiment.copy()})
df = pd.concat([df,df_add],axis=1)

In [ ]:
#Sanity check
df

In [ ]:
with open('fixed_experiments.pkl','wb') as file:
    pkl.dump(df, file)

# Experiment 3: Single-source diffusion
We start with a weight function with w(v) = 0 for all but one vertex; a random vertex gets w(v) = 1

Each round, we pick an edge and replace the vertex weights by the average

We save the dictionary (round -> wasserstein distance between w and uniform)

The first time we run the experiment, we run until the distance is < 1/(20|V|) and we record the number of rounds this took

Then we run every other sample for the recorded number of rounds 

In [ ]:
#This is what gets added to the dataframe
#Note that I save a copy of 'experiment' to the dataframe, so re-initializing it is fine
experiment = {}
        
#Parameters were set at the beginning
num_rolls = rolls['single-source diffusion']

for name in datasets.keys():
    V, E = datasets[name]

    #s determines how often the wasserstein distance is computed
    s = 10

    #Weight function
    w = {v: 0 for v in V}
    #a random node gets weight 1
    w[random.choice(V)] = 1   
    results = diffusion(w, E, 1/(20*len(V)), slice=s)

    #Record the length of the experiment and fix it for the rest
    first_len = len(results)
    for i in range(num_rolls-1):
        w = {v:0 for v in V}
        w[random.choice(V)] = 1
        sample = diffusion(w, E, first_len, slice=s)
        for j in results.keys():
            results[j] += sample[j]
    for j in results.keys():
        results[j] = results[j]/num_rolls

    #Saving
    experiment[name] = results.copy()


    for q in q_list:
        E_bu = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
        
        w = {v: 0 for v in V}
        w[random.choice(V)] = 1   
        results = diffusion(w, E_bu, first_len, slice=s)
        for i in range(num_rolls-1):
            E_bu = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
            w = {v:0 for v in V}
            w[random.choice(V)] = 1
            sample = diffusion(w, E_bu, first_len, slice=s)
            for j in results.keys():
                results[j] += sample[j]
        for j in results.keys():
            results[j] = results[j]/num_rolls
    
        #Saving
        experiment['{0} (q={1})'.format(name, q)] = results.copy()
    
#Saving to the dataframe
df_add = pd.DataFrame({'single-source diffusion':experiment.copy()})
df = pd.concat([df,df_add],axis=1)

In [ ]:
#Sanity check
df

In [ ]:
with open('fixed_experiments.pkl','wb') as file:
    pkl.dump(df, file)

# Experiment 4: 10% sprinkled diffusion
Identical to experiment 3 but 10% of nodes start with w(v) = 1 and we stop when the distance is < 0.005

In both of these experiments, the idea is to reach 5% of the initial distance

In [ ]:
#This is what gets added to the dataframe
#Note that I save a copy of 'experiment' to the dataframe, so re-initializing it is fine
experiment = {}

#Parameters were set at the beginning
num_rolls = rolls['10% sprinkled diffusion']

for name in datasets.keys():
    V,E = datasets[name]

    #s determines how often the wasserstein distance is computed
    s = 10

    #Weight function
    w = {v:0 for v in V}
    #10% of nodes get re-weighted
    V_ini = random.sample(V,k=round(len(V)/10))
    for v in V_ini:
        w[v] = 1 
    results = diffusion(w,E,0.005,slice=s)

    #Record the length of the experiment and fix it for the rest
    first_len = len(results)
    
    for i in range(num_rolls-1):
        w = {v:0 for v in V}
        V_ini = random.sample(V,k=round(len(V)/10))
        for v in V_ini:
            w[v] = 1 
        sample = diffusion(w,E,first_len,slice=s)
        for j in results.keys():
            results[j] += sample[j]
    for j in results.keys():
        results[j] = results[j]/num_rolls

    #Saving
    experiment[name] = results.copy()

    for q in q_list:
        #Bottom-up
        E_bu = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
        
        #Weight function
        w = {v:0 for v in V}
        #10% of nodes get re-weighted
        V_ini = random.sample(V,k=round(len(V)/10))
        for v in V_ini:
            w[v] = 1 
        results = diffusion(w, E_bu, first_len, slice=s)
        for i in range(num_rolls-1):
            E_bu = hm.simplicial_chung_lu(d[name], m[name], q, multisets=multisets, skeleton=skeleton)
            #Weight function
            w = {v:0 for v in V}
            #10% of nodes get re-weighted
            V_ini = random.sample(V,k=round(len(V)/10))
            for v in V_ini:
                w[v] = 1 
            sample = diffusion(w, E_bu, first_len, slice=s)
            for j in results.keys():
                results[j] += sample[j]
        for j in results.keys():
            results[j] = results[j]/num_rolls
    
        #Saving
        experiment['{0} (q={1})'.format(name, q)] = results.copy()
    
#Saving to the dataframe
df_add = pd.DataFrame({'10% sprinkled diffusion':experiment.copy()})
df = pd.concat([df,df_add],axis=1)

In [ ]:
#Sanity check
df

In [ ]:
with open('fixed_experiments.pkl','wb') as file:
    pkl.dump(df, file)

In [ ]:
with open('fixed_experiments.pkl','rb') as file:
    df = pkl.load(file)
df

In [ ]:
t_end = time()
print((t_end - t_start)/60)